# MySQL 테이블 생성 및 데이터 입력 방법

In [3]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [5]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

import pymysql
conn = pymysql.connect(
    host='132.226.20.89',
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
)

## 테이블 생성

In [8]:
cur = conn.cursor()
sql_create_table = '''
    create table if not exists users (
        uid varchar(20) not null primary key,
        pwd char(44) not null,
        uname varchar(20) not null,
        email varchar(40),
        reg_date datetime default current_timestamp,
        is_deleted int default 0
    );
'''

In [9]:
cur.execute(sql_create_table)

0

## 데이터 추가

In [11]:
cur = conn.cursor()
sql_insert = "INSERT INTO users(uid, pwd, uname) VALUES('admin', '1234', '관리자');"
cur.execute(sql_insert)

1

In [13]:
sql_select = 'select * from users;'
cur.execute(sql_select)
row = cur.fetchone()

In [14]:
row

('admin', '1234', '관리자', None, datetime.datetime(2021, 8, 25, 13, 29, 6), 0)

In [15]:
# SQL 캐시로 가지고 있어서 하이디와 연동이 안 됨
# 데이터를 하드 디스크에 쓰게 하는 명령
conn.commit()

- 여러건의 데이터 추가

In [16]:
sql_multi_insert = """
    INSERT INTO users(uid, pwd, uname) VALUES
        ('eskim', '1234', '김은숙'), ('wjlee', '1234', '이우정');
"""

In [17]:
cur.execute(sql_multi_insert)
conn.commit()

- Placeholder

In [18]:
sql_insert_ph = "INSERT INTO users(uid, pwd, uname) VALUES(%s,'1234',%s);"
uid, uname = 'djy', '대조영'
cur.execute(sql_insert_ph, (uid, uname))
conn.commit()

- 여러건의 데이터를 placeholder를 이용해서 insert

In [21]:
# 이런 방법으로도 추가가 가능하지만
users = (('gdhong', '홍길동'),('jbpark', '박재범'))
cur.executemany(sql_insert_ph, users)
conn.commit()

IntegrityError: ignored

In [22]:
# 앞으로는 이 방법을 사용  # 강사님이 추천하신 방법
users = (('gdong2', '홍길동'),('jbpark2','박재범'))
for user in users:
    cur.execute(sql_insert_ph, user)
conn.commit()

IntegrityError: ignored

### 마무리

In [23]:
cur.close()
conn.close()